# Treating database

### Importing database

In [3]:
import json
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer

from ast import literal_eval
from unidecode import unidecode

In [4]:
db_file = open('resume_base.json', 'rb')
db = json.loads(db_file.read())
db[0:3]

[{'id': 1,
  'title': 'Candidato de Contabilidade',
  'age': 49,
  'location': 'Centro, Curitiba - PR',
  'professional_goals_salary': '{"max": 0.0, "min": 0.0}',
  'professional_goals_contract_type': None,
  'professional_goals_working_hours': 'Período Integral',
  'education': '[{"dateEnd": "2003-11-01", "dateInit": "2003-02-01", "courseName": "Curso extra-curricular / Profissionalizante  em Contabilidade para executivos", "educationInstitutionName": "UNICENP em PR"}, {"dateEnd": "1989-12-01", "dateInit": "1985-02-01", "courseName": "Ensino Superior em Ciências contábeis", "educationInstitutionName": "FESP-Fundação de estudos Sociais do Parana em PR"}, {"dateEnd": "1984-11-01", "dateInit": "1982-02-01", "courseName": "Curso Técnico em Contabilidade", "educationInstitutionName": "CEPC-Centro de Estudos Profission.de Curitiba em PR"}]',
  'work_experience': '[{"field": "Contábil, Finanças, Economia", "level": null, "title": "CONTADOR", "dateEnd": "2006-12-01", "dateInit": "2005-03-01",

In [5]:
original_raw_data = pd.DataFrame(db)
raw_data = original_raw_data.copy()
raw_data = raw_data.set_index('id')
raw_data.head()

,age,complementary_data,education,language,location,marital_status,professional_goals_contract_type,professional_goals_salary,professional_goals_working_hours,sex,title,work_experience
id,,,,,,,,,,,,
1,49,"{""travel"": true, ""driving"": ""B"", ""vehicle"": ""C...","[{""dateEnd"": ""2003-11-01"", ""dateInit"": ""2003-0...","[{""level"": ""Intermediário"", ""language"": ""Inglê...","Centro, Curitiba - PR",None,None,"{""max"": 0.0, ""min"": 0.0}",Período Integral,Não Especificado,Candidato de Contabilidade,"[{""field"": ""Contábil, Finanças, Economia"", ""le..."
2,38,"{""travel"": true, ""driving"": ""A"", ""vehicle"": ""C...","[{""dateEnd"": ""2005-12-01"", ""dateInit"": ""2001-0...","[{""level"": ""Avançado"", ""language"": ""Inglês""}, ...",Santa Helena - PR,None,None,"{""max"": 0.0, ""min"": 0.0}",Período Integral,Não Especificado,Candidato de Zootecnia,"[{""field"": ""Agricultura, Pecuária, Veterinária..."
3,54,"{""travel"": true, ""driving"": ""B"", ""vehicle"": nu...","[{""dateEnd"": ""1999-10-01"", ""dateInit"": ""1999-0...","[{""level"": ""Intermediário"", ""language"": ""Inglê...","Centro, Curitiba - PR",None,None,"{""max"": 0.0, ""min"": 0.0}",Período Integral,Não Especificado,Candidato de Hotelaria,"[{""field"": ""Hotelaria, Turismo"", ""level"": null..."
4,41,"{""travel"": false, ""driving"": ""B"", ""vehicle"": ""...","[{""dateEnd"": ""2001-09-01"", ""dateInit"": ""1997-0...","[{""level"": ""Intermediário"", ""language"": ""Inglê...","Centro, Curitiba - PR",None,None,"{""max"": 0.0, ""min"": 0.0}",Período Integral,Não Especificado,Candidato de Marketing,"[{""field"": ""Marketing"", ""level"": null, ""title""..."
5,31,"{""travel"": false, ""driving"": null, ""vehicle"": ...","[{""dateEnd"": ""2004-05-01"", ""dateInit"": ""2004-0...","[{""level"": ""Intermediário"", ""language"": ""Portu...","Jardim Mediterrâneo, Colombo - PR",None,None,"{""max"": 0.0, ""min"": 0.0}",Período Integral,Não Especificado,Candidato de Instrumentação Industrial,"[{""field"": ""Industrial, Produção, Fábrica"", ""l..."


## Treating Data

In [6]:
treated_dataset = pd.DataFrame()
no_treatment_needed_attributes = ['age']
treated_dataset[no_treatment_needed_attributes] = raw_data[no_treatment_needed_attributes]

In [7]:
def swap_none_for_constant(data, column, fill_value, missing_values=None):
    imputer = SimpleImputer(missing_values=missing_values, strategy='constant', fill_value=fill_value)
    imputer.fit(data[column].values.reshape(-1,1))
    data[column] = imputer.transform(data[column].values.reshape(-1,1))

In [26]:
def one_edit_apart(s1, s2):
    if abs(len(s1) - len(s2)) > 1:
        return False
    if len(s1) > len(s2):
        bigger = s1
        smaller = s2
    else:
        bigger = s2
        smaller = s1

    equal_length_strings = len(s1) == len(s2)
    saw_difference = False
    i = 0
    j = 0
    while i < len(smaller): 
        if (bigger[j] != smaller[i]):
            if (saw_difference):
                return False
            saw_difference = True
            if (not equal_length_strings):
                i = i - 1
        i = i + 1
        j = j + 1
    return True

### Professional Goals Salary Treat

For a first approach as it is a litterally numerical attribute we will just divide into a max and min attribute

In [8]:
raw_data['professional_goals_salary'].unique()

array(['{"max": 0.0, "min": 0.0}', None, '{"max": 3000.0, "min": 1200.0}',
       ..., '{"max": 2450.0, "min": 2300.0}',
       '{"max": 14000.0, "min": 7000.0}',
       '{"max": 4800.0, "min": 2800.0}'], dtype=object)

Tem valores de None, entao a primeira abordagem trivial é só substituir por {"max": 0.0, "min":0.0}, pode ser interessante usar a média dos valores futuramente (inclusive substituindo as atuais ocorrencias de {"max": 0.0, "min":0.0})

In [9]:
display(raw_data['professional_goals_salary'].isnull().values.any())
swap_none_for_constant(raw_data, 'professional_goals_salary', {"max": 0.0, "min":0.0})
raw_data['professional_goals_salary'].isnull().values.any()

True

False

In [7]:
treated_dataset['professional_goals_salary_min'] = raw_data['professional_goals_salary'].astype(str).map(lambda v: literal_eval(v)['min'])
treated_dataset['professional_goals_salary_max'] = raw_data['professional_goals_salary'].astype(str).map(lambda v: literal_eval(v)['max'])

In [8]:
treated_dataset[['professional_goals_salary_min', 'professional_goals_salary_max']].sample(5)

,professional_goals_salary_min,professional_goals_salary_max
id,,
577,400.0,750.0
58380,0.0,0.0
1870,400.0,1000.0
85173,1800.0,2000.0
58504,700.0,1000.0


### Professional Goals Contract Type Treat

In [15]:
raw_data['professional_goals_contract_type'].astype(str).apply(unidecode).unique()

array(['Nao Informado', 'Estagio', 'Efetivo - CLT', 'Outros',
       'Temporario', 'Autonomo', 'Trainee', 'Cooperado',
       'Prestador de Servicos (PJ)', 'Estgio', 'Efetivo  CLT'],
      dtype=object)

Necessario unificar termos com distancia de edição até 1 e incluir não informado nas situações com None

In [16]:
swap_none_for_constant(raw_data, 'professional_goals_contract_type', "Nao Informado")

In [18]:
raw_data['professional_goals_contract_type'].astype(str).apply(unidecode).unique()

array(['Nao Informado', 'Estagio', 'Efetivo - CLT', 'Outros',
       'Temporario', 'Autonomo', 'Trainee', 'Cooperado',
       'Prestador de Servicos (PJ)', 'Estgio', 'Efetivo  CLT'],
      dtype=object)

In [19]:
core_contracts_types = ['Nao Informado', 'Estagio', 'Efetivo - CLT', 'Outros',
       'Temporario', 'Autonomo', 'Trainee', 'Cooperado', 'Prestador de Servicos (PJ)']

treated_dataset['professional_goals_contract_type'] = raw_data['professional_goals_contract_type'].astype(str).apply(unidecode)
clean_contract_type = np.array(treated_dataset['professional_goals_contract_type'])

for contract_type in core_contracts_types:
    for i in range(0,len(clean_contract_type)):
        if one_edit_apart(clean_contract_type[i], contract_type):
            clean_contract_type[i] = contract_type

treated_dataset['professional_goals_contract_type'] = clean_contract_type

In [20]:
treated_dataset['professional_goals_contract_type'].unique()

array(['Nao Informado', 'Estagio', 'Efetivo - CLT', 'Outros',
       'Temporario', 'Autonomo', 'Trainee', 'Cooperado',
       'Prestador de Servicos (PJ)'], dtype=object)

### Professional Goals Working Hours

In [21]:
raw_data['professional_goals_working_hours'].astype(str).apply(unidecode).unique()

array(['Periodo Integral', 'Parcial manhas', 'Nao Informado',
       'Parcial tardes', 'Parcial noites', 'Noturno', 'Perodo Integral',
       'Parcial manhs'], dtype=object)

In [22]:
swap_none_for_constant(raw_data, 'professional_goals_working_hours', "Nao Informado")

In [23]:
core_working_hours = ['Periodo Integral', 'Parcial manhas', 'None', 'Parcial tardes',
                       'Parcial noites', 'Noturno']

In [24]:
treated_dataset['professional_goals_working_hours'] = raw_data['professional_goals_working_hours'].astype(str).apply(unidecode)
clean_working_hours = np.array(treated_dataset['professional_goals_working_hours'])

for working_hours in core_working_hours:
    for i in range(0,len(clean_working_hours)):
        if one_edit_apart(clean_working_hours[i], working_hours):
            clean_working_hours[i] = working_hours

treated_dataset['professional_goals_working_hours'] = clean_working_hours

In [25]:
treated_dataset['professional_goals_working_hours'].unique()

array(['Periodo Integral', 'Parcial manhas', 'Nao Informado',
       'Parcial tardes', 'Parcial noites', 'Noturno'], dtype=object)

### Location Treat

In [ ]:
experimentation = raw_data.copy()
experimentation['count'] = 1
experimentation.groupby('location').sum().sort_values('count', ascending=False)

In [ ]:
test_db = [ {'age':resume['age'], 'id': resume['id'], 'professional_goals_working_hours': resume['professional_goals_working_hours'], 'sex':resume['sex'], 'title': resume['title']} for resume in db]

In [ ]:
test_db[0]

In [ ]:
with open('treated_data.json', 'w') as outfile:
    json.dump(test_db, outfile)